In [ ]:
# install pyspark if necessary
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=8576632d6ccb6431e6676296d5d3520c1c1107c75bfb4a21b9825c968d500a7b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# import library needed
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [ ]:
# create a spark session
spark = SparkSession.builder.appName("DataAnalysis").getOrCreate()

# load the dataframe
df_matchs = spark.read.format('csv').options(header = 'True').load('/content/drive/MyDrive/Colab Notebooks/data/football_matches.csv')

# preview of the data
df_matchs.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


Meaning of the features :


*   Match_ID : identifier for each match
*   Div : championship level (D1 = Bundesliga, 1st level, D2 = 2.Bundesliga, 2nd level, E0 = 3rd level
* Season : Starting year of the season
* Date : date of the match
* Home Team : The team that received the game
* Away Team : The team being received by the home team
* FTHG : Goals scored by the home team
* FTAG : Goals scored by the away team
* FTR : Result (H = Home team wins ; A = Away team wins ; D= draw)



In [ ]:
# Rename of 3 last columns
df_matchs = df_matchs.selectExpr(
    "*",
    "`FTHG` AS `HomeTeamGoals`",
    "`FTAG` AS `AwayTeamGoals`",
    "`FTR` AS `Result`"
)

# delete the 3 old columns

df_matchs = df_matchs.drop("FTHG", "FTAG", "FTR")

In [ ]:
# preview of the data
df_matchs.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


In [ ]:
# preview with pandas
df_matchs.toPandas().head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Result
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [ ]:
# create binary column for victory, defeat and draw
df_matchs = df_matchs.withColumn("HomeTeamWin", when(col("Result") == "H", 1).otherwise(0))

df_matchs.toPandas().head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Result,HomeTeamWin
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,1
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,0
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,0
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,1
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,0
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D,0
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H,1
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A,0
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H,1
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H,1


In [ ]:
df_matchs = df_matchs.withColumn("AwayTeamWin", when(col("Result") == "A", 1).otherwise(0)) \
                      .withColumn("GameTie", when(col("Result") == "D", 1).otherwise(0))

# preview with pandas
df_matchs.toPandas().head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Result,HomeTeamWin,AwayTeamWin,GameTie
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,1,0,0
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,0,1,0
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,0,0,1
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,1,0,0
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,0,1,0
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D,0,0,1
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H,1,0,0
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A,0,1,0
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H,1,0,0
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H,1,0,0


In [ ]:
# Create a dataframe only for Bundesliga matches
df_bundesliga = df_matchs.filter((col("Div") == "D1") &
                                (col("Season") >= 2000) &
                                (col("Season") <= 2015))

In [ ]:
# preview with pandas
df_bundesliga.toPandas().head(100)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,Result,HomeTeamWin,AwayTeamWin,GameTie
0,21,D1,2009,2010-02-06,Bochum,Leverkusen,1,1,D,0,0,1
1,22,D1,2009,2009-11-22,Bayern Munich,Leverkusen,1,1,D,0,0,1
2,23,D1,2009,2010-05-08,M'gladbach,Leverkusen,1,1,D,0,0,1
3,24,D1,2009,2009-08-08,Mainz,Leverkusen,2,2,D,0,0,1
4,25,D1,2009,2009-10-17,Hamburg,Leverkusen,0,0,D,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,116,D1,2009,2010-02-20,Mainz,Bochum,0,0,D,0,0,1
96,117,D1,2009,2010-03-06,Wolfsburg,Bochum,4,1,H,1,0,0
97,118,D1,2009,2009-09-12,Hoffenheim,Bochum,3,0,H,1,0,0
98,119,D1,2009,2009-12-19,Hannover,Bochum,2,3,A,0,1,0


In [ ]:
# Create a dataframe containing statistics on Bundesliga home matches from 2000 to 2015
df_bundes_home_matches = df_bundesliga.groupby("Season", "HomeTeam") \
                                      .agg(sum("HomeTeamWin").alias("TotalHomeWin"),
                                           sum("GameTie").alias("TotalHomeTie"),
                                           sum("AwayTeamWin").alias("TotalHomeLoss"),
                                           sum("HomeTeamGoals").alias("HomeScoredGoals"),
                                           sum("AwayTeamGoals").alias("HomeConcededGoals")) \
                                      .withColumnRenamed("HomeTeam", "Team")

# preview with pandas
df_bundes_home_matches.toPandas()

,Season,Team,TotalHomeWin,TotalHomeTie,TotalHomeLoss,HomeScoredGoals,HomeConcededGoals
0,2011,Hamburg,3,7,7,19.0,29.0
1,2005,Kaiserslautern,5,5,7,26.0,33.0
2,2006,Cottbus,6,5,6,21.0,22.0
3,2001,St Pauli,4,4,9,19.0,28.0
4,2005,Mainz,6,7,4,31.0,23.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,7,3,7,21.0,21.0
284,2004,Werder Bremen,9,4,4,33.0,15.0
285,2013,Augsburg,9,3,5,27.0,22.0
286,2004,Stuttgart,12,2,3,34.0,15.0


In [ ]:
# Create a dataframe containing statistics on Bundesliga away matches from 2000 to 2015
df_bundes_away_matches = df_bundesliga.groupby("Season", "AwayTeam") \
                                      .agg(sum("AwayTeamWin").alias("TotalAwayWin"),
                                           sum("GameTie").alias("TotalAwayTie"),
                                           sum("HomeTeamWin").alias("TotalAwayLoss"),
                                           sum("AwayTeamGoals").alias("AwayScoredGoals"),
                                           sum("HomeTeamGoals").alias("AwayConcededGoals")) \
                                      .withColumnRenamed("AwayTeam", "Team")

# preview with pandas
df_bundes_away_matches.toPandas()

,Season,Team,TotalAwayWin,TotalAwayTie,TotalAwayLoss,AwayScoredGoals,AwayConcededGoals
0,2011,Hamburg,5,5,7,16.0,28.0
1,2005,Kaiserslautern,3,4,10,21.0,38.0
2,2006,Cottbus,5,3,9,17.0,27.0
3,2001,St Pauli,0,6,11,18.0,42.0
4,2005,Mainz,3,4,10,15.0,24.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,4,4,9,16.0,28.0
284,2004,Werder Bremen,9,1,7,35.0,22.0
285,2013,Augsburg,6,4,7,20.0,25.0
286,2004,Stuttgart,5,5,7,20.0,25.0


In [ ]:
# join home and away data
df_merged = df_bundes_home_matches.join(df_bundes_away_matches, ["Season", "Team"], "inner")

# preview with pandas
df_merged.toPandas()

,Season,Team,TotalHomeWin,TotalHomeTie,TotalHomeLoss,HomeScoredGoals,HomeConcededGoals,TotalAwayWin,TotalAwayTie,TotalAwayLoss,AwayScoredGoals,AwayConcededGoals
0,2011,Hamburg,3,7,7,19.0,29.0,5,5,7,16.0,28.0
1,2005,Kaiserslautern,5,5,7,26.0,33.0,3,4,10,21.0,38.0
2,2006,Cottbus,6,5,6,21.0,22.0,5,3,9,17.0,27.0
3,2001,St Pauli,4,4,9,19.0,28.0,0,6,11,18.0,42.0
4,2005,Mainz,6,7,4,31.0,23.0,3,4,10,15.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
283,2004,Bielefeld,7,3,7,21.0,21.0,4,4,9,16.0,28.0
284,2004,Werder Bremen,9,4,4,33.0,15.0,9,1,7,35.0,22.0
285,2013,Augsburg,9,3,5,27.0,22.0,6,4,7,20.0,25.0
286,2004,Stuttgart,12,2,3,34.0,15.0,5,5,7,20.0,25.0


In [ ]:
# Create columns for total scores and results
df_total = df_merged.withColumn("TotalGoalsScored", col("HomeScoredGoals") + col("AwayScoredGoals")) \
                .withColumn("TotalGoalsConceded", col("HomeConcededGoals") + col("AwayConcededGoals")) \
                .withColumn("TotalWins", col("TotalHomeWin") + col("TotalAwayWin")) \
                .withColumn("TotalDraws", col("TotalHomeTie") + col("TotalAwayTie")) \
                .withColumn("TotalLosses", col("TotalHomeLoss") + col("TotalAwayLoss"))

# preview with pandas
df_total.toPandas()

,Season,Team,TotalHomeWin,TotalHomeTie,TotalHomeLoss,HomeScoredGoals,HomeConcededGoals,TotalAwayWin,TotalAwayTie,TotalAwayLoss,AwayScoredGoals,AwayConcededGoals,TotalGoalsScored,TotalGoalsConceded,TotalWins,TotalDraws,TotalLosses
0,2011,Hamburg,3,7,7,19.0,29.0,5,5,7,16.0,28.0,35.0,57.0,8,12,14
1,2005,Kaiserslautern,5,5,7,26.0,33.0,3,4,10,21.0,38.0,47.0,71.0,8,9,17
2,2006,Cottbus,6,5,6,21.0,22.0,5,3,9,17.0,27.0,38.0,49.0,11,8,15
3,2001,St Pauli,4,4,9,19.0,28.0,0,6,11,18.0,42.0,37.0,70.0,4,10,20
4,2005,Mainz,6,7,4,31.0,23.0,3,4,10,15.0,24.0,46.0,47.0,9,11,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2004,Bielefeld,7,3,7,21.0,21.0,4,4,9,16.0,28.0,37.0,49.0,11,7,16
284,2004,Werder Bremen,9,4,4,33.0,15.0,9,1,7,35.0,22.0,68.0,37.0,18,5,11
285,2013,Augsburg,9,3,5,27.0,22.0,6,4,7,20.0,25.0,47.0,47.0,15,7,12
286,2004,Stuttgart,12,2,3,34.0,15.0,5,5,7,20.0,25.0,54.0,40.0,17,7,10


In [ ]:
# List of unnecessary columns
cols_to_drop = ['TotalHomeWin', 'TotalHomeTie', 'TotalHomeLoss', 'TotalAwayWin', 'TotalAwayTie', 'TotalAwayLoss', 'HomeScoredGoals', 'HomeConcededGoals', 'AwayScoredGoals', 'AwayConcededGoals']

# Drop unnecessary columns
df_total_cleaned = df_total.drop(*cols_to_drop)

# preview with pandas
df_total_cleaned.toPandas()


,Season,Team,TotalGoalsScored,TotalGoalsConceded,TotalWins,TotalDraws,TotalLosses
0,2011,Hamburg,35.0,57.0,8,12,14
1,2005,Kaiserslautern,47.0,71.0,8,9,17
2,2006,Cottbus,38.0,49.0,11,8,15
3,2001,St Pauli,37.0,70.0,4,10,20
4,2005,Mainz,46.0,47.0,9,11,14
...,...,...,...,...,...,...,...
283,2004,Bielefeld,37.0,49.0,11,7,16
284,2004,Werder Bremen,68.0,37.0,18,5,11
285,2013,Augsburg,47.0,47.0,15,7,12
286,2004,Stuttgart,54.0,40.0,17,7,10


In [ ]:
# Rename of the columns
df_total_cleaned = df_total_cleaned.withColumnRenamed("TotalWins", "Wins") \
                                  .withColumnRenamed("TotalLosses", "Loses") \
                                  .withColumnRenamed("TotalDraws", "Draws") \
                                  .withColumnRenamed("TotalGoalsScored", "GoalsScored") \
                                  .withColumnRenamed("TotalGoalsConceded", "GoalsConceded")

# preview with pandas
df_total_cleaned.toPandas()

,Season,Team,GoalsScored,GoalsConceded,Wins,Draws,Loses
0,2011,Hamburg,35.0,57.0,8,12,14
1,2005,Kaiserslautern,47.0,71.0,8,9,17
2,2006,Cottbus,38.0,49.0,11,8,15
3,2001,St Pauli,37.0,70.0,4,10,20
4,2005,Mainz,46.0,47.0,9,11,14
...,...,...,...,...,...,...,...
283,2004,Bielefeld,37.0,49.0,11,7,16
284,2004,Werder Bremen,68.0,37.0,18,5,11
285,2013,Augsburg,47.0,47.0,15,7,12
286,2004,Stuttgart,54.0,40.0,17,7,10


In [ ]:
# create additional columns for further analysis
df_processed = df_total_cleaned.withColumn("GoalsDifference", col("GoalsScored") - col("GoalsConceded")) \
                                .withColumn("WinPercentage", round((100 * col("Wins") / (col("Wins") + col("Loses") + col("Draws"))), 2))

# preview with pandas
df_processed.toPandas()

,Season,Team,GoalsScored,GoalsConceded,Wins,Draws,Loses,GoalsDifference,WinPercentage
0,2011,Hamburg,35.0,57.0,8,12,14,-22.0,23.53
1,2005,Kaiserslautern,47.0,71.0,8,9,17,-24.0,23.53
2,2006,Cottbus,38.0,49.0,11,8,15,-11.0,32.35
3,2001,St Pauli,37.0,70.0,4,10,20,-33.0,11.76
4,2005,Mainz,46.0,47.0,9,11,14,-1.0,26.47
...,...,...,...,...,...,...,...,...,...
283,2004,Bielefeld,37.0,49.0,11,7,16,-12.0,32.35
284,2004,Werder Bremen,68.0,37.0,18,5,11,31.0,52.94
285,2013,Augsburg,47.0,47.0,15,7,12,0.0,44.12
286,2004,Stuttgart,54.0,40.0,17,7,10,14.0,50.00


Je vais ensuite determiné le classement pour chaque saison en comptant le nombre de points par équipes et en departageant si besoin les équipes par goal average particulier

In [ ]:
# create classement for each season by calculing the points number
df_classement = df_processed.withColumn("TotalPoints", 3 * col("Wins") + col("Draws")) \
                            .withColumn("Rank", row_number().over(Window.partitionBy("Season").orderBy(col("TotalPoints").desc(), col("GoalsDifference").desc())))

# preview with pandas
df_classement.toPandas()

,Season,Team,GoalsScored,GoalsConceded,Wins,Draws,Loses,GoalsDifference,WinPercentage,TotalPoints,Rank
0,2000,Bayern Munich,62.0,37.0,19,6,9,25.0,55.88,63,1
1,2000,Schalke 04,65.0,35.0,18,8,8,30.0,52.94,62,2
2,2000,Dortmund,62.0,42.0,16,10,8,20.0,47.06,58,3
3,2000,Leverkusen,54.0,40.0,17,6,11,14.0,50.00,57,4
4,2000,Hertha,58.0,52.0,18,2,14,6.0,52.94,56,5
...,...,...,...,...,...,...,...,...,...,...,...
283,2015,Werder Bremen,50.0,65.0,10,8,16,-15.0,29.41,38,14
284,2015,Hoffenheim,39.0,54.0,9,10,15,-15.0,26.47,37,15
285,2015,Ein Frankfurt,34.0,52.0,9,9,16,-18.0,26.47,36,16
286,2015,Stuttgart,50.0,75.0,9,6,19,-25.0,26.47,33,17


To finish, we make a list of the champions of each year

In [ ]:
# Create the list of champions
df_champions = df_classement.filter(col("Rank") == 1)

# preview with pandas
df_champions.toPandas()

,Season,Team,GoalsScored,GoalsConceded,Wins,Draws,Loses,GoalsDifference,WinPercentage,TotalPoints,Rank
0,2000,Bayern Munich,62.0,37.0,19,6,9,25.0,55.88,63,1
1,2001,Dortmund,62.0,33.0,21,7,6,29.0,61.76,70,1
2,2002,Bayern Munich,70.0,25.0,23,6,5,45.0,67.65,75,1
3,2003,Werder Bremen,79.0,38.0,22,8,4,41.0,64.71,74,1
4,2004,Bayern Munich,75.0,33.0,24,5,5,42.0,70.59,77,1
5,2005,Bayern Munich,67.0,32.0,22,9,3,35.0,64.71,75,1
6,2006,Stuttgart,61.0,37.0,21,7,6,24.0,61.76,70,1
7,2007,Bayern Munich,68.0,21.0,22,10,2,47.0,64.71,76,1
8,2008,Wolfsburg,80.0,41.0,21,6,7,39.0,61.76,69,1
9,2009,Bayern Munich,72.0,31.0,20,10,4,41.0,58.82,70,1
